Testing multiple hypothesis from the same data can be problematic. The chance of finding false significance can be surprisingly high. 



In [15]:
import pandas as pd
import numpy as np
import numpy.random as nr
import statsmodels.api as sm

import matplotlib.pyplot as plt
%matplotlib inline

To illustrate this, consider applying a t-test to multiple identical Normally distributed variables. In this case, we will create a data set with 20 independently identically distributed (iid) Normal distributions of 1000 samples each.  

In [9]:
ncolumns = 20
nr.seed(234)
normal_vars = nr.normal(size=(1000,ncolumns))
print('The means of the columns are\n', np.mean(normal_vars, axis = 0))
print('\nThe variances of the columns are\n', np.var(normal_vars, axis = 0))

The means of the columns are
 [-1.16191649e-01  2.80829317e-02 -1.78516419e-02 -1.44691489e-02
  3.03718152e-02  1.20007442e-02 -9.58845606e-05  1.98662580e-03
  4.94154934e-02 -4.11640866e-02 -6.32977862e-03 -5.93868192e-02
 -2.56373595e-02  1.43568791e-02 -1.44725765e-02 -1.37023955e-02
  1.80622439e-02  5.87029691e-02 -2.02650514e-02 -1.56346106e-02]

The variances of the columns are
 [0.94834508 1.04744241 1.0258018  0.96977571 1.0089001  1.04113864
 1.00657222 0.99192594 1.04713487 1.04329434 1.04023108 0.96791346
 1.03706907 1.07179865 1.01431404 1.05060289 1.02054329 0.9686211
 1.02810287 0.99521555]


Notice that means and variances are close to 0.0 and 1.0. There is not much difference between these variables. 

Now for each pair of variables we will compute the t-statistic and p-value and append them to lists.

In [10]:
from scipy.stats import ttest_ind, f_oneway
from itertools import product
#ttest_results = np.zeros((ncolumns*ncolumns,2))
ttest_results = []
p_values = []
for i,j in product(range(ncolumns),range(ncolumns)):
    if(i != j): # We only want to test between different samples 
        t1, t2 = ttest_ind(normal_vars[:,i], normal_vars[:,j])
        ttest_results.append(t1)
        p_values.append(t2)

How many of these t-tests would show **significance**. To find out, filter the test results and print results that show significance. 

In [11]:
signifiance_level = 0.05
def find_significant(p_values, ttest_results, signifiance_level):
    n_cases = 0
    for i in range(len(p_values)):
        if(p_values[i] < signifiance_level): 
            n_cases += 1
            print('t-test with SIGNIFICANT, t-statistic = ', round(ttest_results[i],2), ' and p-value = ', round(p_values[i],4))
    print('\nNumber of falsely significant tests = ', n_cases)        
find_significant(p_values, ttest_results, signifiance_level)        

t-test with SIGNIFICANT, t-statistic =  -3.23  and p-value =  0.0013
t-test with SIGNIFICANT, t-statistic =  -2.21  and p-value =  0.0271
t-test with SIGNIFICANT, t-statistic =  -2.32  and p-value =  0.0204
t-test with SIGNIFICANT, t-statistic =  -3.31  and p-value =  0.0009
t-test with SIGNIFICANT, t-statistic =  -2.87  and p-value =  0.0041
t-test with SIGNIFICANT, t-statistic =  -2.62  and p-value =  0.0087
t-test with SIGNIFICANT, t-statistic =  -2.68  and p-value =  0.0074
t-test with SIGNIFICANT, t-statistic =  -3.71  and p-value =  0.0002
t-test with SIGNIFICANT, t-statistic =  -2.46  and p-value =  0.0139
t-test with SIGNIFICANT, t-statistic =  -2.03  and p-value =  0.0424
t-test with SIGNIFICANT, t-statistic =  -2.9  and p-value =  0.0037
t-test with SIGNIFICANT, t-statistic =  -2.29  and p-value =  0.0218
t-test with SIGNIFICANT, t-statistic =  -2.29  and p-value =  0.0221
t-test with SIGNIFICANT, t-statistic =  -3.02  and p-value =  0.0025
t-test with SIGNIFICANT, t-statisti

Notice the large number of apparently significant tests. Do you trust these results to show any important relationships in the data? 

Can the Bonforoni correction help? Let's give it a try?

In [12]:
signifiance_bonforoni = signifiance_level/380.0
print('With Bonforoni correction the significance level is now = ', signifiance_bonforoni)
find_significant(p_values, ttest_results, signifiance_bonforoni)  

With Bonforoni correction the significance level is now =  0.00013157894736842105
t-test with SIGNIFICANT, t-statistic =  -3.99  and p-value =  0.0001
t-test with SIGNIFICANT, t-statistic =  3.99  and p-value =  0.0001

Number of falsely significant tests =  2


Even with the Bonforoni correction we have some false significance tests, if only just barely!

But, can we detect small effect with Bonforoni correction?

In [13]:
nr.seed(567)
ttest_ind(normal_vars[:,0], nr.normal(loc = 0.01, size=(1000,1)))

Ttest_indResult(statistic=array([-2.49553488]), pvalue=array([0.01265684]))

Would ANOVA work better? Let's give it a try for the first 10 columns of the data frame.

In [14]:
f_oneway(normal_vars[:,0],normal_vars[:,1],normal_vars[:,2],normal_vars[:,3],normal_vars[:,4],normal_vars[:,5],normal_vars[:,6],normal_vars[:,7],normal_vars[:,8],normal_vars[:,9])

F_onewayResult(statistic=2.143527277355356, pvalue=0.022915451627654522)